# Ecological Outcome Verification (EOV): Monitoring Land Health

This notebook helps you track and measure the health of your land using the **Savory Institute's Ecological Outcome Verification** framework.

EOV is a science-based method for measuring whether your land management practices are actually improving the soil, water, biodiversity, and overall ecosystem on your farm or ranch. It was developed by the Savory Institute as part of their Land to Market program, which connects regenerative producers to buyers who value verified ecological outcomes.

**Why does this matter?**

- It gives you hard numbers to prove your land is improving (or flag problems early)
- It qualifies you for the Land to Market seal, opening premium market access
- It builds a documented record for grant applications and conservation programs
- It connects your holistic management decisions to measurable results on the ground

Whether you are just starting holistic management or have been doing it for years, EOV gives you a consistent way to answer the question: **Is my land getting healthier?**

## How to Run This Notebook

1. Make sure you have **uv** installed (see [uv docs](https://docs.astral.sh/uv/))
2. From the project root, install dependencies with the data extras:
   ```bash
   uv sync --extra data
   ```
3. Start Jupyter:
   ```bash
   uv run jupyter notebook
   ```
4. Open this notebook and run cells in order from top to bottom
5. Replace the sample data with your own farm or ranch data where indicated

Each section explains what the code does before you run it. You do not need programming experience to follow along.

## Setup

First we load the tools this notebook needs. Run this cell once at the start.

In [ ]:
# Standard library
from datetime import datetime

# Data handling
import pandas as pd

try:
    import matplotlib.colors as mcolors
    import matplotlib.pyplot as plt

    HAS_MATPLOTLIB = True
    plt.rcParams["figure.figsize"] = (10, 6)
    plt.rcParams["figure.dpi"] = 100
except ImportError:
    HAS_MATPLOTLIB = False
    print("Optional: pip install matplotlib for charts")

try:
    from tabulate import tabulate

    HAS_TABULATE = True
except ImportError:
    HAS_TABULATE = False
    print("Optional: pip install tabulate for prettier tables")

# Configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

print(f"Notebook ready - {datetime.now().strftime('%B %d, %Y')}")

## What is EOV?

**Ecological Outcome Verification (EOV)** is a monitoring protocol developed by the [Savory Institute](https://savory.global/) to measure whether land management practices are leading to genuine ecological improvement.

### Key concepts

- **Holistic Management** is the decision-making framework. EOV is how you measure whether those decisions are working.
- **Land to Market** is the Savory Institute's verification program. Producers who demonstrate positive ecological trends through EOV monitoring can earn the Land to Market seal for their products.
- **Ecological Health Index (EHI)** is the overall score calculated from 15 individual indicators measured at each monitoring site.

### How it works in practice

1. You establish fixed monitoring sites on your property
2. At each site, you assess 15 ecological indicators (we will cover each one below)
3. Each indicator gets a score from -2 (severely degraded) to +2 (thriving)
4. Scores are summed into an overall EHI score ranging from -30 to +30
5. You repeat assessments seasonally to track trends over time

The power of EOV is in the **trend**. A single assessment tells you where you stand today. Repeated assessments over seasons and years show whether your land is on an upward or downward trajectory.

## The 15 Ecological Health Indicators

Each indicator is scored on a 5-point scale from **-2** (severely degraded) to **+2** (excellent/thriving). A score of **0** means stable but not improving. The table below lists every indicator with a plain-language description and what each score level means.

In [ ]:
# Build the reference table of all 15 EOV indicators
raw_indicators = pd.DataFrame({
    "indicator": [
        "Living Organisms",
        "Biological Crust",
        "Litter Cover",
        "Litter Incorporation",
        "Soil Surface Roughness",
        "Water Infiltration",
        "Mineral Cycle",
        "Compaction",
        "Soil Structure",
        "Soil Erosion",
        "Plant Diversity",
        "Plant Spacing",
        "Plant Vigor",
        "Canopy Cover",
        "Ground Cover",
    ],
    "description": [
        "Insects, earthworms, dung beetles, and other visible soil life",
        "Mosses, lichens, and algae crusts on the soil surface",
        "Dead plant material lying on the soil surface",
        "How well old litter is being broken down into the soil",
        "Physical texture and unevenness of the soil surface",
        "How quickly water soaks into the ground vs running off",
        "Nutrient cycling from decomposing organic matter",
        "How hard-packed the soil is (inverted: less compaction = higher score)",
        "Crumb structure, aggregation, and pore spaces in the soil",
        "Signs of sheet, rill, or gully erosion (inverted: less erosion = higher score)",
        "Variety of plant species present at the site",
        "Gaps between plants (inverted: smaller gaps = higher score)",
        "Overall health, color, and growth energy of plants",
        "Percentage of sky blocked by plant leaves above ground level",
        "Percentage of soil surface covered by living plants at ground level",
    ],
    "score_neg2": [
        "No visible organisms",
        "No crust present, bare soil",
        "No litter, bare ground",
        "No decomposition occurring",
        "Completely smooth, sealed surface",
        "Water pools and runs off immediately",
        "No decomposition, grey/pale soil",
        "Rock-hard, cannot push in probe",
        "Massive, blocky, no aggregation",
        "Active gully or severe sheet erosion",
        "Monoculture or single species",
        "Bare ground >75% between plants",
        "Plants dying or severely stressed",
        "<10% canopy coverage",
        "<10% ground covered by plants",
    ],
    "score_zero": [
        "Some organisms but low activity",
        "Patchy crust, partial coverage",
        "Thin litter in some areas",
        "Slow breakdown, litter piling up",
        "Moderate texture, some crusting",
        "Slow infiltration, some ponding",
        "Slow cycling, moderate organic matter",
        "Moderate resistance to probe",
        "Some aggregation, some blocky areas",
        "Minor rills, stable but not healing",
        "3-5 species, moderate variety",
        "Moderate spacing, 25-50% bare",
        "Average health, some stress signs",
        "40-60% canopy coverage",
        "40-60% ground coverage",
    ],
    "score_pos2": [
        "Abundant worms, beetles, insects",
        "Continuous healthy crust layer",
        "Thick, even litter coverage",
        "Rapid decomposition into humus",
        "Rough, porous, well-structured surface",
        "Water soaks in immediately",
        "Active cycling, dark rich soil",
        "Loose, easy probe penetration",
        "Excellent crumb structure throughout",
        "No erosion, signs of healing",
        "8+ species, high diversity",
        "Dense, <10% bare ground",
        "Lush, vigorous, deep green growth",
        ">80% canopy coverage",
        ">80% ground coverage",
    ],
})

print("EOV Ecological Health Indicators - Scoring Rubric")
print("=" * 60)
print()
if HAS_TABULATE:
    print(tabulate(
        raw_indicators[["indicator", "description"]],
        headers=["Indicator", "What to Look For"],
        tablefmt="simple",
        showindex=range(1, 16),
    ))
else:
    print(raw_indicators[["indicator", "description"]].to_string())

### Scoring Guide at a Glance

Here is what the scores from -2 to +2 look like for each indicator. When you are out in the field, use this as a quick reference.

In [ ]:
# Show scoring examples for each indicator
scoring_view = raw_indicators[[
    "indicator", "score_neg2", "score_zero", "score_pos2"
]].copy()
scoring_view.columns = ["Indicator", "Score = -2 (Degraded)", "Score = 0 (Stable)", "Score = +2 (Thriving)"]

if HAS_TABULATE:
    print(tabulate(
        scoring_view,
        headers="keys",
        tablefmt="grid",
        showindex=False,
        maxcolwidths=[20, 30, 30, 30],
    ))
else:
    print(scoring_view.to_string())

## Recording a Site Assessment

When you visit a monitoring site, you will score each of the 15 indicators. Below is an example assessment. **Replace the sample scores with your own field observations.**

In [ ]:
# Record a single site assessment
# Replace these values with your actual field observations
raw_assessment = pd.DataFrame([{
    "date": "2026-02-15",
    "site_name": "North Pasture - Hilltop",
    "assessor": "Maria Gonzalez",
    "living_organisms": 1,
    "biological_crust": 0,
    "litter_cover": 1,
    "litter_incorporation": 1,
    "soil_surface_roughness": 0,
    "water_infiltration": 1,
    "mineral_cycle": 1,
    "compaction": -1,
    "soil_structure": 0,
    "soil_erosion": 0,
    "plant_diversity": 1,
    "plant_spacing": 1,
    "plant_vigor": 2,
    "canopy_cover": 1,
    "ground_cover": 1,
}])

print("Site Assessment Recorded")
print(f"  Site: {raw_assessment['site_name'].iloc[0]}")
print(f"  Date: {raw_assessment['date'].iloc[0]}")
print(f"  Assessor: {raw_assessment['assessor'].iloc[0]}")
print()

### Viewing the scores

Let's look at the individual indicator scores for this assessment.

In [ ]:
# List all 15 indicator columns
indicator_columns = [
    "living_organisms", "biological_crust", "litter_cover",
    "litter_incorporation", "soil_surface_roughness", "water_infiltration",
    "mineral_cycle", "compaction", "soil_structure", "soil_erosion",
    "plant_diversity", "plant_spacing", "plant_vigor",
    "canopy_cover", "ground_cover",
]

# Display scores in a readable format
scores_display = pd.DataFrame({
    "Indicator": [col.replace("_", " ").title() for col in indicator_columns],
    "Score": [raw_assessment[col].iloc[0] for col in indicator_columns],
})

if HAS_TABULATE:
    print(tabulate(scores_display, headers="keys", tablefmt="simple", showindex=False))
else:
    print(scores_display.to_string())

## Calculating the EHI Score

The **Ecological Health Index (EHI)** is simply the sum of all 15 indicator scores. Because each indicator ranges from -2 to +2, the total EHI ranges from **-30** (completely degraded) to **+30** (thriving ecosystem).

| EHI Range | Interpretation |
|-----------|----------------|
| +20 to +30 | Excellent - land is thriving |
| +10 to +19 | Good - clear signs of health |
| +1 to +9 | Fair - some improvement, room to grow |
| 0 | Neutral - stable but not improving |
| -1 to -9 | Concerning - early signs of degradation |
| -10 to -19 | Poor - significant degradation |
| -20 to -30 | Critical - severe land health crisis |

In [ ]:
# Calculate the Ecological Health Index
ehi_score = raw_assessment[indicator_columns].sum(axis=1).iloc[0]

# Interpret the score
if ehi_score >= 20:
    interpretation = "Excellent - your land is thriving"
elif ehi_score >= 10:
    interpretation = "Good - clear signs of ecological health"
elif ehi_score >= 1:
    interpretation = "Fair - some improvement, room to grow"
elif ehi_score == 0:
    interpretation = "Neutral - stable but not yet improving"
elif ehi_score >= -9:
    interpretation = "Concerning - early signs of degradation"
elif ehi_score >= -19:
    interpretation = "Poor - significant degradation present"
else:
    interpretation = "Critical - severe land health crisis"

print(f"EHI Score: {ehi_score} out of 30")
print(f"Interpretation: {interpretation}")
print(f"\nPercentage of maximum: {((ehi_score + 30) / 60) * 100:.0f}%")

## Multi-Season Trend Visualization

The real power of EOV is tracking changes over time. Below we build sample data for three monitoring sites assessed across four seasons. **Replace this sample data with your actual assessments.**

In [ ]:
# Sample multi-season data for 3 sites
# Replace with your actual seasonal assessment data
raw_seasonal = pd.DataFrame([
    {"site": "North Pasture", "season": "Spring 2025", "ehi_score": 4},
    {"site": "North Pasture", "season": "Summer 2025", "ehi_score": 7},
    {"site": "North Pasture", "season": "Fall 2025",   "ehi_score": 10},
    {"site": "North Pasture", "season": "Winter 2026", "ehi_score": 9},
    {"site": "Creek Bottom",  "season": "Spring 2025", "ehi_score": 12},
    {"site": "Creek Bottom",  "season": "Summer 2025", "ehi_score": 15},
    {"site": "Creek Bottom",  "season": "Fall 2025",   "ehi_score": 14},
    {"site": "Creek Bottom",  "season": "Winter 2026", "ehi_score": 16},
    {"site": "South Ridge",   "season": "Spring 2025", "ehi_score": -3},
    {"site": "South Ridge",   "season": "Summer 2025", "ehi_score": -1},
    {"site": "South Ridge",   "season": "Fall 2025",   "ehi_score": 2},
    {"site": "South Ridge",   "season": "Winter 2026", "ehi_score": 5},
])

print("Seasonal EHI scores loaded for 3 sites across 4 seasons.")
if HAS_TABULATE:
    print(tabulate(raw_seasonal, headers="keys", tablefmt="simple", showindex=False))
else:
    print(raw_seasonal.to_string())

### EHI Trend Line Chart

This chart shows how each site's overall health score has changed over the seasons. An upward line means the land is improving.

In [ ]:
# Line chart: EHI score over time per site
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))

    seasons = ["Spring 2025", "Summer 2025", "Fall 2025", "Winter 2026"]
    colors = ["#2ecc71", "#3498db", "#e67e22"]

    for i, site in enumerate(raw_seasonal["site"].unique()):
        site_data = raw_seasonal[raw_seasonal["site"] == site]
        ax.plot(site_data["season"], site_data["ehi_score"],
                marker="o", linewidth=2, label=site, color=colors[i])

    ax.axhline(y=0, color="gray", linestyle="--", alpha=0.5, label="Neutral (0)")
    ax.set_title("Ecological Health Index - Seasonal Trends", fontsize=14)
    ax.set_ylabel("EHI Score (-30 to +30)")
    ax.set_xlabel("Season")
    ax.legend(loc="lower right")
    ax.set_ylim(-10, 25)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

### Indicator Heatmap

A heatmap lets you see all 15 indicators at once for a single site, colored from red (degraded) to green (thriving). This makes it easy to spot which indicators need attention.

In [ ]:
# Build detailed indicator data for North Pasture across seasons
raw_heatmap = pd.DataFrame({
    "Living Organisms":       [0, 1, 1, 1],
    "Biological Crust":       [-1, 0, 0, 0],
    "Litter Cover":           [0, 1, 1, 1],
    "Litter Incorporation":   [0, 0, 1, 1],
    "Surface Roughness":      [0, 0, 1, 0],
    "Water Infiltration":     [1, 1, 1, 1],
    "Mineral Cycle":          [0, 1, 1, 1],
    "Compaction":             [-1, -1, 0, 0],
    "Soil Structure":         [0, 0, 0, 1],
    "Soil Erosion":           [0, 0, 1, 0],
    "Plant Diversity":        [1, 1, 1, 1],
    "Plant Spacing":          [1, 1, 1, 1],
    "Plant Vigor":            [1, 1, 1, 1],
    "Canopy Cover":           [1, 1, 0, 0],
    "Ground Cover":           [1, 0, 0, 0],
}, index=["Spring 2025", "Summer 2025", "Fall 2025", "Winter 2026"])

print("North Pasture - indicator scores by season:")
print(raw_heatmap)

Now we plot the heatmap so you can visually scan for patterns.

In [ ]:
# Heatmap of indicator scores
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(14, 5))

    cmap = plt.cm.RdYlGn  # Red = bad, Yellow = neutral, Green = good
    norm = mcolors.Normalize(vmin=-2, vmax=2)

    im = ax.imshow(raw_heatmap.values, cmap=cmap, norm=norm, aspect="auto")

    ax.set_xticks(range(len(raw_heatmap.columns)))
    ax.set_xticklabels(raw_heatmap.columns, rotation=45, ha="right", fontsize=9)
    ax.set_yticks(range(len(raw_heatmap.index)))
    ax.set_yticklabels(raw_heatmap.index)
    ax.set_title("North Pasture - Ecological Indicator Heatmap", fontsize=14)

    # Add score values inside each cell
    for i in range(len(raw_heatmap.index)):
        for j in range(len(raw_heatmap.columns)):
            val = raw_heatmap.values[i, j]
            ax.text(j, i, f"{val:+d}", ha="center", va="center",
                    fontsize=10, fontweight="bold")

    plt.colorbar(im, ax=ax, label="Score (-2 to +2)", shrink=0.8)
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

## Soil and Water Infiltration Benchmarking

Water infiltration is one of the most important indicators of soil health. How quickly water soaks into your soil tells you a lot about compaction, organic matter, and biological activity.

The test is simple: pour a known amount of water on the ground and time how long it takes to soak in. Below we compare your results against regional benchmarks.

In [ ]:
# Infiltration test results (inches per hour)
# Replace with your actual field measurements
raw_infiltration = pd.DataFrame({
    "site": ["North Pasture", "Creek Bottom", "South Ridge",
             "Benchmark: Healthy Grassland", "Benchmark: Degraded"],
    "infiltration_rate_in_per_hr": [2.5, 4.1, 0.8, 3.0, 0.5],
    "category": ["Your Sites", "Your Sites", "Your Sites",
                 "Benchmark", "Benchmark"],
})

print("Water Infiltration Rates (inches per hour)")
print("=" * 50)
if HAS_TABULATE:
    print(tabulate(
        raw_infiltration[["site", "infiltration_rate_in_per_hr"]],
        headers=["Site", "Infiltration (in/hr)"],
        tablefmt="simple",
        showindex=False,
    ))
else:
    print(raw_infiltration[["site", "infiltration_rate_in_per_hr"]].to_string())

### Infiltration Rate Comparison Chart

This bar chart compares your sites against the healthy grassland and degraded benchmarks. Sites above the healthy benchmark line are performing well; sites below the degraded line need attention.

In [ ]:
# Bar chart comparing infiltration rates
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))

    bar_colors = ["#2ecc71" if cat == "Your Sites" else "#95a5a6"
                  for cat in raw_infiltration["category"]]

    bars = ax.bar(raw_infiltration["site"],
                  raw_infiltration["infiltration_rate_in_per_hr"],
                  color=bar_colors, edgecolor="white", linewidth=0.5)

    ax.axhline(y=3.0, color="green", linestyle="--", alpha=0.7,
               label="Healthy grassland benchmark")
    ax.axhline(y=0.5, color="red", linestyle="--", alpha=0.7,
               label="Degraded benchmark")

    ax.set_title("Water Infiltration Rate by Site", fontsize=14)
    ax.set_ylabel("Infiltration Rate (inches/hour)")
    ax.legend()
    ax.grid(True, axis="y", alpha=0.3)
    plt.xticks(rotation=15, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Install matplotlib to see this chart")

## Photo Point Documentation Protocol

Photographs are a critical part of EOV monitoring. A photo taken from the same spot, same direction, at the same time of year creates a powerful visual record of change.

### Setting up a photo point

1. **Mark the spot** with a permanent stake or T-post
2. **Record GPS coordinates** (use your phone or a handheld GPS)
3. **Choose a compass bearing** (e.g., face due North) and always shoot the same direction
4. **Take photos at the same time of year** to avoid seasonal appearance differences confusing the comparison
5. **Include a reference object** in the frame (the stake, a yardstick) for scale

Below is a log template for tracking your photo points.

In [ ]:
# Photo point documentation log
raw_photo_log = pd.DataFrame([
    {"photo_point_id": "PP-001", "site_name": "North Pasture - Hilltop",
     "gps_lat": 38.8951, "gps_lon": -77.0364, "compass_bearing": "North (0)",
     "established_date": "2025-03-15", "last_photo_date": "2026-01-20",
     "photos_taken": 4, "notes": "T-post with orange cap as marker"},
    {"photo_point_id": "PP-002", "site_name": "Creek Bottom - East Bank",
     "gps_lat": 38.8940, "gps_lon": -77.0350, "compass_bearing": "East (90)",
     "established_date": "2025-03-15", "last_photo_date": "2026-01-20",
     "photos_taken": 4, "notes": "White PVC stake, include creek in frame"},
    {"photo_point_id": "PP-003", "site_name": "South Ridge - Midslope",
     "gps_lat": 38.8930, "gps_lon": -77.0370, "compass_bearing": "South (180)",
     "established_date": "2025-04-01", "last_photo_date": "2026-01-21",
     "photos_taken": 3, "notes": "Rebar with pink flagging tape"},
])

print("Photo Point Documentation Log")
print("=" * 50)
if HAS_TABULATE:
    print(tabulate(
        raw_photo_log[["photo_point_id", "site_name", "compass_bearing",
                        "photos_taken", "last_photo_date"]],
        headers=["ID", "Site", "Bearing", "Photos", "Last Photo"],
        tablefmt="simple",
        showindex=False,
    ))
else:
    print(raw_photo_log[["photo_point_id", "site_name", "compass_bearing",
                          "photos_taken", "last_photo_date"]].to_string())

## EOV Enrollment Readiness Checklist

If you are considering enrolling in the Savory Institute's Land to Market program, here is what you need to have in place. This checklist helps you assess your readiness.

In [ ]:
# EOV enrollment readiness checklist
raw_checklist = pd.DataFrame([
    {"requirement": "Minimum 2 established monitoring sites",
     "status": "Complete", "notes": "3 sites established March 2025"},
    {"requirement": "At least 1 year of baseline monitoring data",
     "status": "In Progress", "notes": "4 seasonal assessments by Spring 2026"},
    {"requirement": "Photo points established at each monitoring site",
     "status": "Complete", "notes": "3 photo points with 3-4 photos each"},
    {"requirement": "Water infiltration tests conducted",
     "status": "Complete", "notes": "Tested all 3 sites"},
    {"requirement": "Assessor trained in EOV protocol",
     "status": "Complete", "notes": "Maria completed Savory training Aug 2025"},
    {"requirement": "Holistic Management plan documented",
     "status": "In Progress", "notes": "Draft plan, needs grazing chart"},
    {"requirement": "Consistent positive ecological trend",
     "status": "In Progress", "notes": "All 3 sites trending upward"},
    {"requirement": "Application submitted to Land to Market",
     "status": "Not Started", "notes": "Target: Summer 2026"},
])

# Color-code the status
status_emoji = {"Complete": "[DONE]", "In Progress": "[WIP]", "Not Started": "[TODO]"}
raw_checklist["progress"] = raw_checklist["status"].map(status_emoji)

print("EOV Enrollment Readiness")
print("=" * 60)
if HAS_TABULATE:
    print(tabulate(
        raw_checklist[["progress", "requirement", "notes"]],
        headers=["Status", "Requirement", "Notes"],
        tablefmt="simple",
        showindex=False,
        maxcolwidths=[8, 45, 40],
    ))
else:
    print(raw_checklist[["progress", "requirement", "notes"]].to_string())

done_count = (raw_checklist["status"] == "Complete").sum()
total_count = len(raw_checklist)
print(f"\nProgress: {done_count}/{total_count} requirements complete")

## Connecting EOV Data to Grant Narratives

Your EOV monitoring data is valuable beyond the Land to Market program. Many conservation grants, USDA programs (like EQIP and CSP), and state-level regenerative agriculture programs ask for documented evidence of ecological outcomes.

**How to use this data in grant applications:**

- **Baseline documentation**: Your initial EHI scores establish where you started
- **Trend data**: Multi-season assessments show improvement trajectory
- **Specific metrics**: Infiltration rates, plant diversity counts, and ground cover percentages give reviewers concrete numbers
- **Photo evidence**: Before/after photo point images are powerful visual proof
- **Indicator detail**: If a grant focuses on water quality, highlight your water infiltration and soil erosion scores specifically

When writing your grant narrative, you might say:

> *Over four seasonal monitoring periods, our North Pasture site improved its Ecological Health Index from +4 to +9, with notable gains in litter incorporation (+0 to +1) and mineral cycling (+0 to +1). Water infiltration rates at this site measure 2.5 inches per hour, approaching the healthy grassland benchmark of 3.0 inches per hour.*

For more guidance on structuring grant applications, see the [Grant Writing for Farms](grant-writing-for-farms.ipynb) notebook in this project.

## Conclusion

EOV gives you a structured, repeatable way to measure whether your land management is working. The key is consistency: monitor the same sites, the same way, season after season.

### Key Takeaways

- The 15 ecological indicators cover soil, water, and plant health
- Each indicator is scored from -2 to +2, with the sum giving your EHI score
- Trends matter more than any single score
- Photo points and infiltration tests add powerful supporting evidence
- This data feeds directly into Land to Market enrollment and grant applications

### Next Steps

- [ ] Walk your property and identify 2-3 representative monitoring sites
- [ ] Establish photo points at each site with GPS coordinates and compass bearings
- [ ] Conduct your first full 15-indicator assessment at each site
- [ ] Run water infiltration tests at each site
- [ ] Schedule seasonal reassessments on your calendar (same time each season)
- [ ] Replace the sample data in this notebook with your own observations
- [ ] Review the Savory Institute training materials for detailed scoring guidance
- [ ] When you have 4 seasonal assessments, evaluate Land to Market enrollment readiness